In [1]:
import base64
import folium
import glob
import io
import os

from IPython.display import display
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

In [2]:
DATA_DIR = 'example'
IMG_EXT = '.jpg'

GPS_EXIF_TAG = 'GPSInfo'
GPS_LAT_TAG = 'GPSLatitude'
GPS_LONG_TAG = 'GPSLongitude'

JERUSALEM_COORD = [31.7683, 35.2137]
DEFAULT_ZOOM = 7
POPUP_IMAGE_SIZE = (200, 100) 
POPUP_MARGIN = 50

In [3]:
def get_images_in_dir(dirname):
    glob_expr = os.path.join(dirname, '*{}'.format(IMG_EXT))
    img_files = glob.glob(glob_expr)
    return img_files
    
def load_images(img_files):
    images = []
    for img_file in img_files:
        img = Image.open(img_file)
        images.append(img)
    return images

def get_gps_img_data(exif_data):
    for tag_id in exif_data:
        # get the tag name, instead of human unreadable tag id
        tag = TAGS.get(tag_id)
        if tag == GPS_EXIF_TAG:
            data = exif_data.get(tag_id)
            return data
    return None

def dms_to_dec(dms):
    # currently handling only N and E
    degree, minute, second = dms
    return int(degree) + float(minute) / 60 + float(second) / 3600

def gps_info_to_coord(gps_info):
    for tag_id, value in gps_info.items():
        tag = GPSTAGS.get(tag_id)
        if tag == GPS_LAT_TAG:
            lat = dms_to_dec(value)
        elif tag == GPS_LONG_TAG:
            long = dms_to_dec(value)
    return lat, long

def get_images_coords(images):
    coords = []
    for img in images:
        exif_data = img.getexif()
        gps_data = get_gps_img_data(exif_data)
        if gps_data is not None and len(gps_data) > 0:
            lat, long = gps_info_to_coord(gps_data)
            coords.append((lat, long))
        else:
            # no gps data found
            coords.append((None, None))
    return coords

In [4]:
image_files = get_images_in_dir(DATA_DIR)
images = load_images(image_files)

In [5]:
coords = get_images_coords(images)

In [6]:
def add_image_marker(ind, img, coord):
    resized = img.resize(POPUP_IMAGE_SIZE)
    html = '<figure>'
    img_byte_arr = io.BytesIO()
    resized.save(img_byte_arr, format=img.format)
    encoded = base64.b64encode(img_byte_arr.getvalue())
    html += '<img src="data:image/jpeg;base64,{}">'.format(encoded.decode('UTF-8'))
    html += '<figcaption>{}</figcaption></figure>'.format(ind)
    width = POPUP_IMAGE_SIZE[0] + POPUP_MARGIN
    height = POPUP_IMAGE_SIZE[1] + POPUP_MARGIN
    iframe = folium.IFrame(html, width=width, height=height)
    tooltip = folium.Popup(iframe)
    return folium.Marker(coord,popup=tooltip)

In [7]:
fmap = folium.Map(location=JERUSALEM_COORD, zoom_start=DEFAULT_ZOOM)
for i, (img, coord) in enumerate(zip(images, coords)):
    if coord[0] is not None:
        marker = add_image_marker(i, img, coord)
        marker.add_to(fmap)
fmap